In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from pandas.plotting import scatter_matrix
from sklearn import svm

In [2]:
datas = pd.read_excel('input/train.xls')

In [3]:
print(datas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 10 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   id                                             1887 non-null   int64 
 1   icerik                                         1887 non-null   object
 2   Gecikti veya Dağıtıma Çıkmadı                  1887 non-null   int64 
 3   Evde yok notu düşüldü veya Kapıya Getirilmedi  1887 non-null   int64 
 4   Telefonlara Cevap Verilmedi                    1887 non-null   int64 
 5   İade Süreci                                    1887 non-null   int64 
 6   Teslim Alınmadı veya Teslim Edilmedi           1887 non-null   int64 
 7   Kötü Diyalog Veya Saygısız Tutum               1887 non-null   int64 
 8   Hasarlı veya Kayıp Paket                       1887 non-null   int64 
 9   Hijyen Kurallarına Uyulmadı                    1887 non-null   

In [4]:
datas['icerik'][:5]

0    Sürat Kargo Gülnur Korkmaz İşitme Engelliyim--...
1    MNG Kargo Teslimat Yapmamak İçin Ellerinden Ge...
2    PTT Kargo Covid Önlemleri Alınmıyor--> PTT Kar...
3    PTT Kargo Ürünü Adrese Getirmeme, Müşteri Hizm...
4    Aras Kargo Kargom Yola Çıkmıyor--> Şubeye gitt...
Name: icerik, dtype: object

In [5]:
X_train, X_test, y_train, y_test = train_test_split(datas.loc[:,'icerik'],datas.drop(['id','icerik'],axis=1), random_state=0)

In [6]:
X_train.shape

(1415,)

In [7]:
y_train.shape

(1415, 8)

In [8]:
X_test.shape

(472,)

In [9]:
y_test.shape

(472, 8)

In [10]:
knn=KNeighborsClassifier(n_neighbors=1)

In [11]:
z=[0,1,2,3,4,5,6,7]
z[0]=datas.loc[:,'Gecikti veya Dağıtıma Çıkmadı']
z[1]=datas.loc[:,'Evde yok notu düşüldü veya Kapıya Getirilmedi']
z[2]=datas.loc[:,'Telefonlara Cevap Verilmedi']
z[3]=datas.loc[:,'İade Süreci']
z[4]=datas.loc[:,'Teslim Alınmadı veya Teslim Edilmedi']
z[5]=datas.loc[:,'Kötü Diyalog Veya Saygısız Tutum']
z[6]=datas.loc[:,'Hasarlı veya Kayıp Paket']
z[7]=datas.loc[:,'Hijyen Kurallarına Uyulmadı']

In [12]:
sikayetsayisi=len(datas)
yontem=CountVectorizer

In [ ]:
tks = '[A-Za-z0-9]+(?=\\s+)'


pl = Pipeline([
        ('vec', yontem(token_pattern = tks)),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

pl.fit(X_train,y_train)

In [ ]:
test = datas.fillna("")
predictions = pl.predict_proba(test.icerik)

In [ ]:
toplam=[0,0,0,0,0,0,0,0]
ortalama=[0,0,0,0,0,0,0,0]
for sutun in range (0,len(z)):
    for i in range (0,len(predictions)):
        toplam[sutun]=toplam[sutun]+predictions[i][sutun]
    ortalama[sutun]=toplam[sutun]/len(predictions)

In [ ]:
for i in range (0 , len(predictions)):
    for j in range (0,len(z)):
        if predictions[i][j] < ortalama[j]:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1

In [ ]:
# Format predictions in DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=y_test.columns, index=test.id, data=predictions)

prediction_df.to_excel('predictions.xls')

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)